In [1]:
from torchsummary import summary
from efficientnet_pytorch import EfficientNet

import os
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import time
import copy

from torch.optim import lr_scheduler
from torchvision import datasets
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
torch.cuda.empty_cache() 

In [3]:
torch.manual_seed(42)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device    ', device)

device     cuda:0


In [5]:

NUM_CLASSES = 5

model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=5)

Loaded pretrained weights for efficientnet-b4


In [6]:
print(model._fc)

Linear(in_features=1792, out_features=5, bias=True)


In [7]:
batch_size = 8
epochs = 50
data_dir = './dataset'

In [8]:
writer = SummaryWriter('./runs/experiment1/')

In [9]:
data_transforms = {'train': transforms.Compose([
    # transforms.Resize(300,300),
#     transforms.RandomRotation(30),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
    'val': transforms.Compose([
        # transforms.Resize(300,300),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#         transforms.Resize(200,200)
    ])}

In [10]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

In [11]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 3818
     Root location: ./dataset/train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 1906
     Root location: ./dataset/val
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [12]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}

In [17]:
print(dataloaders['train'])

In [16]:
dataloaders['val']

In [18]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [19]:
class_names = image_datasets['train'].classes
class_names

['1_CH_NG', '1_DOUBLE_NG', '1_PO_NG', '1_SEAL_NG', 'OK']

In [20]:
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [23]:
import torchsummary

In [27]:
torchsummary.summary(model,(3,300,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 301, 301]               0
Conv2dStaticSamePadding-2         [-1, 48, 150, 150]           1,296
       BatchNorm2d-3         [-1, 48, 150, 150]              96
MemoryEfficientSwish-4         [-1, 48, 150, 150]               0
         ZeroPad2d-5         [-1, 48, 152, 152]               0
Conv2dStaticSamePadding-6         [-1, 48, 150, 150]             432
       BatchNorm2d-7         [-1, 48, 150, 150]              96
MemoryEfficientSwish-8         [-1, 48, 150, 150]               0
          Identity-9             [-1, 48, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 12, 1, 1]             588
MemoryEfficientSwish-11             [-1, 12, 1, 1]               0
         Identity-12             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 48, 1, 1]             624
         I

In [28]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [29]:
since = time.time()

In [30]:
best_model_weights = copy.deepcopy(model.state_dict())
best_acc = 0.0

In [ ]:
 for model_num, epoch in enumerate(range(epochs), start=1):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()

            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                # print(labels)
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                # print(running_loss)
                running_corrects += torch.sum(preds == labels.data)
 
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            # writer.add_graph('epoch loss', epoch_loss, epoch)
            # writer.add_graph('epoch acc', epoch_acc, epoch)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss, epoch_acc))

            # if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            model_weights = copy.deepcopy(model.state_dict())
            # print(best_acc)
        torch.save(model_weights, f'./weights/model_{model_num}.pth')
        #torch.cuda.empty_cache() 

  0%|          | 0/478 [00:00<?, ?it/s]

Epoch 0/49
----------


  0%|          | 0/239 [00:00<?, ?it/s]

train Loss: 0.0886 Acc: 0.9783


  0%|          | 0/478 [00:00<?, ?it/s]

...

  0%|          | 0/239 [00:00<?, ?it/s]

train Loss: 0.0009 Acc: 0.9997


  0%|          | 0/478 [00:00<?, ?it/s]

val Loss: 0.0010 Acc: 0.9995
Epoch 45/49
----------


 91%|█████████ | 434/478 [00:57<00:05,  7.72it/s]